
# $K^* \mu \mu$ angular analysis - Getting started

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from iminuit import Minuit

## Loading the data

Taking sig.pkl and splitting into bins using bit from Voltron V4

In [2]:
df = pd.read_pickle("data/sig.pkl")

# %% bin splitting
# I'm sure there's a more elegant solution but I can't think of one right now.

bin0 = df[(df['q2'] > 0.1) & (df['q2'] <= 0.98)]
bin1 = df[(df['q2'] > 1.1) & (df['q2'] <= 2.5)]
bin2 = df[(df['q2'] > 2.5) & (df['q2'] <= 4.0)]
bin3 = df[(df['q2'] > 4.0) & (df['q2'] <= 6.0)]
bin4 = df[(df['q2'] > 6.0) & (df['q2'] <= 8.0)]
bin5 = df[(df['q2'] > 15.0) & (df['q2'] <= 17.0)]
bin6 = df[(df['q2'] > 17.0) & (df['q2'] <= 19.0)]
bin7 = df[(df['q2'] > 11.0) & (df['q2'] <= 12.5)]
bin8 = df[(df['q2'] > 1.0) & (df['q2'] <= 6.0)] # ???
# we're only given toy data up to bin 6, so how can we check that this isn't a mistake?
bin9 = df[(df['q2'] > 15.0) & (df['q2'] <= 17.9)]

bins = [bin0, bin1, bin2, bin3, bin4, bin5, bin6, bin7, bin8, bin9]

bins[0].head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/sig.pkl'

Let us have a look at the distribution of the variable $cos(\theta_l)$, which will be used below, in one of the bins.

## Fitting
Let us assume a flat acceptance distribution for the following.
We have
$\frac{d^2\Gamma_P}{dq^2dcos\theta_l} = \frac{3}{8}[\frac{3}{2} - \frac{1}{2}F_L + \frac{1}{2}cos2\theta_l(1-3F_L) + \frac{8}{3}A_{FB}cos\theta_l]$

Let us check how the negative log-likelihood function defined above looks like for a given bin and a fixed $A_{FB}$ or a fixed $F_L$.

### Fitting
We then use `iminuit` to optimize $A_{FB}$ and $F_L$.

Let us also look at the profiles for $A_{FB}$ and $F_L$ for a bin (e.g. bin number 0).

We can also plot the resulting fitted PDF over the measured $cos(\theta_l)$ distribution.

Finally, let us visualize the results in each bin.

## $\theta_L$

In [ ]:
#THETAL
plt.hist(bins[3]['costhetal'], bins=25, density=True)
plt.xlabel(r'$cos(\theta_l)$')
plt.ylabel(r'Number of candidates')
plt.grid()
plt.show()

In [ ]:
#THETAL
def d2gamma_p_d2q2_dcosthetal(fl, afb, cos_theta_l):
    """
    Returns the pdf defined above
    :param fl: f_l observable
    :param afb: a_fb observable
    :param cos_theta_l: cos(theta_l)
    :return:
    """
    ctl = cos_theta_l
    c2tl = 2 * ctl ** 2 - 1
    acceptance = 0.5  # acceptance "function"
    scalar_array = 3/8 * (3/2 - 1/2 * fl + 1/2 * c2tl * (1 - 3 * fl) + 8/3 * afb * ctl) * acceptance
    normalised_scalar_array = scalar_array * 2  # normalising scalar array to account for the non-unity acceptance function
    return normalised_scalar_array



In [ ]:
#THETAL
def log_likelihoodthetal(fl, afb, _bin): #FOR THETA_L
    """
    Returns the negative log-likelihood of the pdf defined above
    :param fl: f_l observable
    :param afb: a_fb observable
    :param _bin: number of the bin to fit
    :return:
    """
    _bin = bins[int(_bin)]
    ctl = _bin['costhetal']
    normalised_scalar_array = d2gamma_p_d2q2_dcosthetal(fl=fl, afb=afb, cos_theta_l=ctl)
    return - np.sum(np.log(normalised_scalar_array))

In [ ]:
_test_bin = 1  #FOR THETA_L
_test_afb = 0.7
_test_fl = 0.0

x = np.linspace(-1, 1, 500)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
ax1.plot(x, [log_likelihoodthetal(fl=i, afb=_test_afb, _bin=_test_bin) for i in x])
ax1.set_title(r'$A_{FB}$ = ' + str(_test_afb))
ax1.set_xlabel(r'$F_L$')
ax1.set_ylabel(r'$-\mathcal{L}$')
ax1.grid()
ax2.plot(x, [log_likelihoodthetal(fl=_test_fl, afb=i, _bin=_test_bin) for i in x])
ax2.set_title(r'$F_{L}$ = ' + str(_test_fl))
ax2.set_xlabel(r'$A_{FB}$')
ax2.set_ylabel(r'$-\mathcal{L}$')
ax2.grid()
plt.tight_layout()
plt.show()

In [ ]:
#FOR THETA_L
bin_number_to_check = 0  # bin that we want to check in more details in the next cell
bin_results_to_check = None

log_likelihoodthetal.errordef = Minuit.LIKELIHOOD
decimal_places = 3
starting_point = [-0.1,0.0]
fls, fl_errs = [], []
afbs, afb_errs = [], []
for i in range(len(bins)):
    m = Minuit(log_likelihoodthetal, fl=starting_point[0], afb=starting_point[1], _bin=i)
    m.fixed['_bin'] = True  # fixing the bin number as we don't want to optimize it
    m.limits=((-1.0, 1.0), (-1.0, 1.0), None)
    m.migrad()
    m.hesse()
    if i == bin_number_to_check:
        bin_results_to_check = m
    fls.append(m.values[0])
    afbs.append(m.values[1])
    fl_errs.append(m.errors[0])
    afb_errs.append(m.errors[1])
    print(f"Bin {i}: {np.round(fls[i], decimal_places)} pm {np.round(fl_errs[i], decimal_places)},", f"{np.round(afbs[i], decimal_places)} pm {np.round(afb_errs[i], decimal_places)}. Function minimum considered valid: {m.fmin.is_valid}")

In [ ]:
#FOR THETA_L
plt.figure(figsize=(8, 5))
plt.subplot(221)
bin_results_to_check.draw_mnprofile('afb', bound=3)
plt.subplot(222)
bin_results_to_check.draw_mnprofile('fl', bound=3)
plt.tight_layout()
plt.show()

In [ ]:
#THETA L
bin_to_plot = 3
number_of_bins_in_hist = 25
cos_theta_l_bin = bins[bin_to_plot]['costhetal']
hist, _bins, _ = plt.hist(cos_theta_l_bin, bins=number_of_bins_in_hist)
x = np.linspace(-1, 1, number_of_bins_in_hist)
pdf_multiplier = np.sum(hist) * (np.max(cos_theta_l_bin) - np.min(cos_theta_l_bin)) / number_of_bins_in_hist
y = d2gamma_p_d2q2_dcosthetal(fl=fls[bin_to_plot], afb=afbs[bin_to_plot], cos_theta_l=x) * pdf_multiplier
plt.plot(x, y, label=f'Fit for bin {bin_to_plot}')
plt.xlabel(r'$cos(\theta_l)$')
plt.ylabel(r'Number of candidates')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#THETAL
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3))
ax1.errorbar(np.linspace(0, len(bins) - 1, len(bins)), fls, yerr=fl_errs, fmt='o', markersize=2, label=r'$F_L$', color='red')
ax2.errorbar(np.linspace(0, len(bins) - 1, len(bins)), afbs, yerr=afb_errs, fmt='o', markersize=2, label=r'$A_{FB}$', color='red')
ax1.grid()
ax2.grid()
ax1.set_ylabel(r'$F_L$')
ax2.set_ylabel(r'$A_{FB}$')
ax1.set_xlabel(r'Bin number')
ax2.set_xlabel(r'Bin number')
plt.tight_layout()
plt.show()


## $\phi$

In [ ]:
#PHI
plt.hist(bins[6]['phi'], bins=25, density=True)
plt.xlabel(r'$\phi$')
plt.ylabel(r'Number of candidates')
plt.grid()
plt.show()

In [ ]:
#PHI
def d2gamma_p_d2q2_dphi(s3, s9, phi):
    """
    Returns the pdf defined above
    :param s3: s3 observable
    :param s9: s9 observable
    :param phi: phi
    :return:
    """

    acceptance = 0.5  # acceptance "function"
    scalar_array = 1/(2*np.pi)*(1+s3*np.cos(2*phi)+s9*np.sin(2*phi)) * acceptance
    normalised_scalar_array = scalar_array * 2  # normalising scalar array to account for the non-unity acceptance function
    return normalised_scalar_array


In [ ]:
#PHI
def log_likelihoodphi(s3, s9, _bin): #FOR PHI
    """
    Returns the negative log-likelihood of the pdf defined above
    :param fl: f_l observable
    :param afb: a_fb observable
    :param _bin: number of the bin to fit
    :return:
    """
    _bin = bins[int(_bin)]
    Phi = _bin['phi']
    normalised_scalar_array = d2gamma_p_d2q2_dphi(s3=s3, s9=s9, phi=Phi)
    return - np.sum(np.log(normalised_scalar_array))

In [ ]:
_test_bin = 6  #FOR PHI
_test_s3 = 0.7
_test_s9 = 0

x = np.linspace(-1, 1, 500)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
ax1.plot(x, [log_likelihoodphi(s3=i, s9=_test_s9, _bin=_test_bin) for i in x])
ax1.set_title(r'$S_9$ = ' + str(_test_s9))
ax1.set_xlabel(r'$S_3$')
ax1.set_ylabel(r'$-\mathcal{L}$')
ax1.grid()
ax2.plot(x, [log_likelihoodphi(s3=_test_s3, s9=i, _bin=_test_bin) for i in x])
ax2.set_title(r'$S_3$ = ' + str(_test_s3))
ax2.set_xlabel(r'$S_9$')
ax2.set_ylabel(r'$-\mathcal{L}$')
ax2.grid()
plt.tight_layout()
plt.show()

In [ ]:
#FOR PHI
bin_number_to_check = 6  # bin that we want to check in more details in the next cell
bin_results_to_check = None

log_likelihoodphi.errordef = Minuit.LIKELIHOOD
decimal_places = 3
starting_point = [-0.25,0.0]
s3s, s3_errs = [], []
s9s, s9_errs = [], []
for i in range(len(bins)):
    m = Minuit(log_likelihoodphi, s3=starting_point[0], s9=starting_point[1], _bin=i)
    m.fixed['_bin'] = True  # fixing the bin number as we don't want to optimize it
    m.limits=((-1, 1), (-1, 1), None)
    m.migrad()
    m.hesse()
    if i == bin_number_to_check:
        bin_results_to_check = m
    s3s.append(m.values[0])
    s9s.append(m.values[1])
    s3_errs.append(m.errors[0])
    s9_errs.append(m.errors[1])
    print(f"Bin {i}: S3={np.round(s3s[i], decimal_places)} pm {np.round(s3_errs[i], decimal_places)},", f" S9={np.round(s9s[i], decimal_places)} pm {np.round(s9_errs[i], decimal_places)}. Function minimum considered valid: {m.fmin.is_valid}")

In [ ]:
#FOR PHI
plt.figure(figsize=(8, 5))
plt.subplot(221)
bin_results_to_check.draw_mnprofile('s9', bound=3)
plt.subplot(222)
bin_results_to_check.draw_mnprofile('s3', bound=3)
plt.tight_layout()
plt.show()

In [ ]:
# PHI
bin_to_plot = 6
number_of_bins_in_hist = 25
phi_bin = bins[bin_to_plot]['phi']
hist, _bins, _ = plt.hist(phi_bin, bins=number_of_bins_in_hist)
x = np.linspace(-np.pi, np.pi, number_of_bins_in_hist)
pdf_multiplier = np.sum(hist) * (np.max(phi_bin) - np.min(phi_bin)) / number_of_bins_in_hist
y = d2gamma_p_d2q2_dphi(s3=s3s[bin_to_plot], s9=s9s[bin_to_plot], phi=x) * pdf_multiplier
plt.plot(x, y, label=f'Fit for bin {bin_to_plot}')
plt.xlabel(r'$\phi$')
plt.ylabel(r'Number of candidates')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#PHI
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3))
ax1.errorbar(np.linspace(0, len(bins) - 1, len(bins)), s3s, yerr=s3_errs, fmt='o', markersize=2, label=r'$S_3$', color='red')
ax2.errorbar(np.linspace(0, len(bins) - 1, len(bins)), s9s, yerr=s9_errs, fmt='o', markersize=2, label=r'$S_9$', color='red')
ax1.grid()
ax2.grid()
ax1.set_ylabel(r'$S_3$')
ax2.set_ylabel(r'$S_9$')
ax1.set_xlabel(r'Bin number')
ax2.set_xlabel(r'Bin number')
plt.tight_layout()
plt.show()

## $\theta_K$

In [ ]:
#THETAK
plt.hist(bins[1]['costhetak'], bins=25, density=True)
plt.xlabel(r'$\cos(\theta_k)$')
plt.ylabel(r'Number of candidates')
plt.grid()
plt.show()

In [ ]:
#THETAK
def d2gamma_p_d2q2_dcosthetak(fl, cos_theta_k):
    """
    Returns the pdf defined above
    :param fl: f_l observable
    :param afb: a_fb observable
    :param cos_theta_k: cos(theta_k)
    :return:
    """
    ctk = cos_theta_k
    acceptance = 0.5  # acceptance "function"
    scalar_array = 3/4 * ((3*fl-1)*ctk**2 +(1-fl)) * acceptance
    normalised_scalar_array = scalar_array * 2  # normalising scalar array to account for the non-unity acceptance function
    return normalised_scalar_array


In [ ]:
def log_likelihoodthetak(fl, _bin): #FOR THETA_K
    """
    Returns the negative log-likelihood of the pdf defined above
    :param fl: f_l observable
    :param afb: a_fb observable
    :param _bin: number of the bin to fit
    :return:
    """
    _bin = bins[int(_bin)]
    ctk = _bin['costhetak']
    normalised_scalar_array = d2gamma_p_d2q2_dcosthetak(fl=fl, cos_theta_k=ctk)
    return - np.sum(np.log(normalised_scalar_array))

In [ ]:
_test_bin = 1  #FOR THETA_K


x = np.linspace(-1, 1, 500)
plt.plot(x, [log_likelihoodthetak(fl=i, _bin=_test_bin) for i in x])
plt.xlabel(r'$F_L$')
plt.ylabel(r'$-\mathcal{L}$')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
#FOR THETA_L
bin_number_to_check = 0  # bin that we want to check in more details in the next cell
bin_results_to_check = None

log_likelihoodthetak.errordef = Minuit.LIKELIHOOD
decimal_places = 3
starting_point = [0.5]
fls, fl_errs = [], []

for i in range(len(bins)):
    m = Minuit(log_likelihoodthetak, fl=starting_point[0], _bin=i)
    m.fixed['_bin'] = True  # fixing the bin number as we don't want to optimize it
    m.limits=((-1.0, 1.0), None)
    m.migrad()
    m.hesse()
    if i == bin_number_to_check:
        bin_results_to_check = m
    fls.append(m.values[0])
    fl_errs.append(m.errors[0])
    print(f"Bin {i}: {np.round(fls[i], decimal_places)} pm {np.round(fl_errs[i], decimal_places)}. Function minimum considered valid: {m.fmin.is_valid}")

In [ ]:
#FOR THETA_K
plt.figure(figsize=(8, 5))
plt.subplot(222)
bin_results_to_check.draw_mnprofile('fl', bound=3)
plt.tight_layout()
plt.show()

In [ ]:
#THETA K
bin_to_plot = 3
number_of_bins_in_hist = 25
cos_theta_k_bin = bins[bin_to_plot]['costhetak']
hist, _bins, _ = plt.hist(cos_theta_l_bin, bins=number_of_bins_in_hist)
x = np.linspace(-1, 1, number_of_bins_in_hist)
pdf_multiplier = np.sum(hist) * (np.max(cos_theta_k_bin) - np.min(cos_theta_k_bin)) / number_of_bins_in_hist
y = d2gamma_p_d2q2_dcosthetak(fl=fls[bin_to_plot], cos_theta_k=x) * pdf_multiplier
plt.plot(x, y, label=f'Fit for bin {bin_to_plot}')
plt.xlabel(r'$cos(\theta_k)$')
plt.ylabel(r'Number of candidates')
plt.legend()
plt.grid()
plt.show()

In [ ]:
#THETAK
fig, (ax1) = plt.subplots(1, figsize=(5, 4))
ax1.errorbar(np.linspace(0, len(bins) - 1, len(bins)), fls, yerr=fl_errs, fmt='o', markersize=2, label=r'$F_L$', color='red')
ax1.grid()
ax1.set_ylabel(r'$F_L$')
ax1.set_xlabel(r'Bin number')

plt.tight_layout()
plt.show()